In [1]:
import geopandas as gpd
import folium
from folium.features import Choropleth
from folium.plugins import MarkerCluster
import os
import io, requests
import zipfile
import joblib
from tqdm import tqdm

# data_path = 'C:/Users/demo/Desktop/Lattitude/datas/'
data_path = 'datas'
os.makedirs(data_path, exist_ok=True)


# Types de voitures

In [2]:
file_name = 'voitures-par-commune-par-energie'
ext = '.geojson'
url = 'http://opendata.agenceore.fr/explore/dataset/voitures-par-commune-par-energie/download?format=geojson&timezone=Europe/Berlin&use_labels_for_header=false'

try: 
    cars = gpd.read_feather(os.path.join(data_path,file_name + '.feather'))
except:
    # from url
    s = requests.get(url).content
    cars = gpd.read_file(io.StringIO(s.decode('utf-8')))

    # from local file
    # cars = gpd.read_file(os.path.join(data_path,file_name + ext))
   
    # avoiding multi dowloads for GES emmision lowering
    cars.to_feather(os.path.join(data_path,file_name + '.feather'))

In [3]:
cars.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 316612 entries, 0 to 316611
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype   
---  ------                   --------------   -----   
 0   nb_vp                    316612 non-null  int64   
 1   libepci                  314625 non-null  object  
 2   libgeo                   316612 non-null  object  
 3   nb_vp_rechargeables_gaz  316612 non-null  int64   
 4   date_arrete              316612 non-null  object  
 5   epci                     314625 non-null  object  
 6   nb_vp_rechargeables_el   316612 non-null  int64   
 7   codgeo                   316612 non-null  object  
 8   geometry                 0 non-null       geometry
dtypes: geometry(1), int64(3), object(5)
memory usage: 21.7+ MB


In [4]:
cars.isna().any()

nb_vp                      False
libepci                     True
libgeo                     False
nb_vp_rechargeables_gaz    False
date_arrete                False
epci                        True
nb_vp_rechargeables_el     False
codgeo                     False
geometry                    True
dtype: bool

In [5]:
# cars.libepci # manque des valeurs

In [6]:
cars.libgeo # complet mais a croiser avec les noms de communes

0                   INCHY
1                   INCHY
2                   INCHY
3                    IWUY
4                    IWUY
               ...       
316607          JONCREUIL
316608          JONCREUIL
316609    JULLY-SUR-SARCE
316610    JULLY-SUR-SARCE
316611    JULLY-SUR-SARCE
Name: libgeo, Length: 316612, dtype: object

In [7]:
cars.codgeo   ## LE BON CLIENT

0         59321
1         59321
2         59321
3         59322
4         59322
          ...  
316607    10180
316608    10180
316609    10181
316610    10181
316611    10181
Name: codgeo, Length: 316612, dtype: object

In [8]:
cars.drop(columns='geometry', inplace=True)

In [9]:
cars['dept'] = cars['codgeo'].apply(lambda s : s[0:2])

# Découpage 


Juste pour poser un marqueur sur la commune

A remplacer avec decoupage administratif 

In [10]:
file_name2 = 'communes-20220101'
ext2 = '.shp'
url ='https://www.data.gouv.fr/fr/datasets/r/0e117c06-248f-45e5-8945-0e79d9136165'
temp_path = 'temp_unzip'


try: 
    communes = gpd.read_feather(os.path.join(data_path,file_name2 + '.feather'))
except:
    # Zip file from url  
    zip_file = requests.get(url).content
    os.makedirs(temp_path, exist_ok=True)
    with zipfile.ZipFile(io.BytesIO(zip_file)) as archive:
        archive.extractall(temp_path)
    communes = gpd.read_file(os.path.join(temp_path,file_name2 + ext2))   
    shutil.rmtree(temp_path) 
    
    communes.to_feather(os.path.join(data_path,file_name2 + '.feather'))


In [11]:
# file_name2 = 'communes-20220101'
# ext2 = '.shp'
# url ='https://www.data.gouv.fr/fr/datasets/r/0e117c06-248f-45e5-8945-0e79d9136165'
# temp_path = 'temp_unzip'

# communes = gpd.read_file(os.path.join(temp_path,file_name2 + ext2)) 
# communes.to_feather(os.path.join(data_path,file_name2 + '.feather'))

In [12]:
communes.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 34955 entries, 0 to 34954
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   insee      34955 non-null  object  
 1   nom        34955 non-null  object  
 2   wikipedia  34951 non-null  object  
 3   surf_ha    34955 non-null  float64 
 4   geometry   34955 non-null  geometry
dtypes: float64(1), geometry(1), object(3)
memory usage: 1.3+ MB


In [13]:
communes.isna().any()

insee        False
nom          False
wikipedia     True
surf_ha      False
geometry     False
dtype: bool

In [14]:
(communes.insee.value_counts() > 1).all()  # no duplicates

False

# Jointure des tables

In [15]:
communes.rename(columns={'insee': 'codgeo_'}, inplace=True)

In [16]:
communes['dept'] = communes['codgeo_'].apply(lambda s : s[0:2])

# Agrégations

In [17]:
car_agg = cars.groupby('codgeo').agg(
                    {'nb_vp': ['min','max'],
                    'nb_vp_rechargeables_gaz': ['min','max'],
                    'nb_vp_rechargeables_el': ['min','max'],
                    'dept': ['first'],
                    # 'nom': ['first'],
                    }
                    ).reset_index()
car_agg.columns = car_agg.columns.to_flat_index()
car_agg.columns = [ x + '_' + y if x !='geometry' else x for x,y in car_agg.columns]
car_agg.rename(columns={'dept_first': 'dept'}, inplace=True)
car_agg

,codgeo_,nb_vp_min,nb_vp_max,nb_vp_rechargeables_gaz_min,nb_vp_rechargeables_gaz_max,nb_vp_rechargeables_el_min,nb_vp_rechargeables_el_max,dept
0,01001,881,921,0,0,6,11,01
1,01002,278,285,0,0,1,4,01
2,01004,14881,15490,0,0,108,257,01
3,01005,1998,2130,0,0,12,28,01
4,01006,152,160,0,0,0,0,01
...,...,...,...,...,...,...,...,...
35184,97699,7,12,0,0,0,0,97
35185,97701,4,6,0,0,1,1,97
35186,97797,1,1,0,0,0,0,97
35187,97801,1,2,0,0,0,0,97


In [18]:
car_agg['pct_el'] = car_agg['nb_vp_rechargeables_el_max'] /  car_agg['nb_vp_max']

In [19]:
display(car_agg.head(0), car_agg.shape)

,codgeo_,nb_vp_min,nb_vp_max,nb_vp_rechargeables_gaz_min,nb_vp_rechargeables_gaz_max,nb_vp_rechargeables_el_min,nb_vp_rechargeables_el_max,dept,pct_el


(35189, 9)

In [20]:
communes['type'] = [ g.geom_type for g in communes.geometry]
communes['type'].value_counts()

Polygon         34621
MultiPolygon      334
Name: type, dtype: int64

In [21]:
display(communes.head(0), communes.shape)

,codgeo_,nom,wikipedia,surf_ha,geometry,dept,type


(34955, 7)

In [22]:
# # Choose only one department

communes.query("(dept == '75') and (dept == '77' ) and (dept == '78') and (dept == '91') and (dept == '92') and (dept == '93') and (dept == '94') and (dept == '95')")


,codgeo_,nom,wikipedia,surf_ha,geometry,dept,type


In [23]:
# Choose only MultiPolygon

communes = communes.query("type == 'Polygon' ")

# Markers

In [24]:
from branca.colormap import linear, LinearColormap

min = car_agg['pct_el'].min()
max = car_agg['pct_el'].max()

# Define the color map
colors = ['#ff0000', '#ffff00', '#00ff00']
cmap_vp = LinearColormap(colors=colors, vmin=min, vmax=max)
display(cmap_vp)

In [25]:
map = folium.Map(location= [48.8, 2.3],
                    width='100%', height='100%', left='00%', top='0%', position='relative', 
                    tiles= 'OpenStreetMap',     #OpenStreetMap', Stamen
                    attr=None,    #  None   Terrain, Toner, and Watercolor
                    min_zoom=0, max_zoom=18, zoom_start=10, 
                    min_lat=- 90, max_lat=90, min_lon=- 180, max_lon=180, max_bounds=False, 
                    crs='EPSG3857', control_scale=True, prefer_canvas=False, 
                    no_touch=False, disable_3d=True, png_enabled=False, zoom_control=True)

In [26]:
# fig1 = folium.FeatureGroup(name='Véhicules par communes',overlay=False).add_to(map)

In [27]:

# custom_scale1 = (car_agg['pct_el'].quantile((0,0.2,0.4,0.6,0.8,1))).tolist()

# folium.Choropleth(
#     geo_data=communes,
#     name='choropleth',
#     data=car_agg,
#     columns=['codgeo_', 'pct_el'],
#     key_on='feature.properties.codgeo_',
#     threshold_scale=custom_scale1, #use the custom scale we created for legend
#     fill_color='YlOrRd',
#     nan_fill_color="White", #Use white color if there is no data available for the county
#     fill_opacity=0.7,
#     line_opacity=0.2,
#     legend_name='Véhicules particuliers ',
#     highlight=True,
#     overlay=False,
#     line_color='black',
#     ).add_to(map)


In [28]:
custom_scale = (car_agg['pct_el'].quantile((0,0.2,0.4,0.6,0.8,1))).tolist()

# Define a function to filter data by department
def filter_by_department(department= '75'):
    filtered_car = car_agg[car_agg['dept'] == department]
    filetred_com = communes[communes['dept'] == department]
    
    c = folium.Choropleth(
        geo_data=filetred_com,
        name='filtered_choropleth',
        data=filtered_car,
        columns=['codgeo_', 'pct_el'],
        key_on='feature.properties.codgeo_',
        # threshold_scale=custom_scale, #use the custom scale we created 
        # fill_color=YlOrRd',
        nan_fill_color="White", #Use white color if there is no data available for the county
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='Véhicules particuliers pct elect ',
        highlight=True,
        overlay=False,
        line_color='black',
    )
    for key in c._children:
        if key.startswith('color_map'):
            del(c._children[key])
    c.add_to(map)

In [29]:
departement_list = communes['dept'].unique().tolist()
for dept in tqdm(departement_list[30:70]):
    filter_by_department(dept)



100%|██████████| 40/40 [00:45<00:00,  1.14s/it]


In [30]:
# add layer control
folium.LayerControl().add_to(map)

# Display the map
map      #.show_in_browser()

Wall time: 0 ns


--------------

In [ ]:
*****************************************************************************************

In [ ]:
nb_vp = MarkerCluster(name='Nombre de véhicules par communes')
nb_el = MarkerCluster(name='Nombre de véhicules électriques')

for idx, row in car_agg.iterrows():
    # Clusters for vp_max circles
    nb_vp.add_child( folium.CircleMarker(
                            location = [row.geometry.y, row.geometry.x],
                            radius = 50,
                            popup = f'<b>véhicules: </b> {row["nb_vp_max"]}',
                            color = cmap_vp(row['nb_vp_max']),
                            fill = True,
                            fill_color = cmap_vp(row['nb_vp_max']),
                            fillOpacity = .1,
                      )
                    )    
    nb_el.add_child( folium.Marker(
                            location = [row.geometry.y, row.geometry.x],
                            popup = f'<b>électriques: </b> {row["nb_vp_rechargeables_el_max"]}',
                            icon = folium.Icon(icon="info-sign", color='blue'),
                      )
                    )


In [ ]:
marker_path = 'datas/markers'
os.makedirs(marker_path, exist_ok=True)

joblib.dump(nb_vp, os.path.join(marker_path,'nombre_vehicules.mrkr'))

joblib.dump(nb_el, os.path.join(marker_path,'nombre_vehicules_electriques.mrkr'))


# Carte

In [ ]:
map = folium.Map(location= [48.8, 2.3],
                    width='100%', height='100%', left='00%', top='0%', position='relative', 
                    tiles= 'OpenStreetMap',     #OpenStreetMap', Stamen
                    attr=None,    #  None   Terrain, Toner, and Watercolor
                    min_zoom=0, max_zoom=18, zoom_start=10, 
                    min_lat=- 90, max_lat=90, min_lon=- 180, max_lon=180, max_bounds=False, 
                    crs='EPSG3857', control_scale=False, prefer_canvas=False, 
                    no_touch=False, disable_3d=False, png_enabled=False, zoom_control=True)

nb_vp.add_to(map)
nb_el.add_to(map)
# trafic.add_to(all_in)   
           
folium.LayerControl().add_to(map)

map

In [ ]:
maps_path = 'datas/maps'
os.makedirs(maps_path, exist_ok=True)


map.save(os.path.join(maps_path,'carte_vehicules.html'))